In [ ]:
import pandas as pd
import json
from typing import Dict, List, Any

# Define custom types for better readability
CountryData = Dict[str, Any]
StadiumData = Dict[str, Any]

def load_and_clean_csv(file_path: str) -> pd.DataFrame:
    """
    Load CSV file and clean the data by removing extra spaces and quotes.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    df = pd.read_csv(file_path, header=None)
    return df.applymap(lambda x: str(x).strip().replace("'", "") if isinstance(x, str) else x)

def load_all_data() -> Dict[str, pd.DataFrame]:
    """
    Load all CSV files and return them as a dictionary of DataFrames.

    Returns:
        Dict[str, pd.DataFrame]: Dictionary containing all loaded DataFrames.
    """
    file_names = ['Country', 'Players', 'Match_results', 'Player_Cards', 'Player_Assists_Goals', 'Worldcup_History']
    return {name: load_and_clean_csv(f'{name}.csv') for name in file_names}

def assign_column_names(data: Dict[str, pd.DataFrame]) -> None:
    """
    Assign column names to each DataFrame in the data dictionary.

    Args:
        data (Dict[str, pd.DataFrame]): Dictionary of DataFrames.
    """
    column_names = {
        'Country': ['Country_Name', 'Population', 'No_of_Worldcup_won', 'Manager', 'Capital'],
        'Players': ['Player_id', 'Full_Name', 'Fname', 'Lname', 'DOB', 'Country', 'Height', 'Club', 'Position', 'Caps_for_country', 'Is_captain'],
        'Match_results': ['Match_id', 'Date', 'Start_time', 'Team1', 'Team2', 'Team1Score', 'Team2Score', 'Stadium', 'Host_city'],
        'Player_Cards': ['Player_id', 'No_of_Yellow_cards', 'No_of_Red_cards'],
        'Player_Assists_Goals': ['Player_id', 'No_of_Matches', 'Goals', 'Assists', 'Minutes_Played'],
        'Worldcup_History': ['Year', 'Host', 'Winner']
    }
    for name, columns in column_names.items():
        data[name].columns = columns

def format_population(population: str) -> float:
    """
    Format population data by removing commas and converting to float.

    Args:
        population (str): Population as a string.

    Returns:
        float: Formatted population.
    """
    try:
        return round(float(str(population).replace(',', '').strip()), 1)
    except ValueError:
        return 0.0

def get_player_stats(player_id: str, cards_data: pd.DataFrame, stats_data: pd.DataFrame) -> Dict[str, int]:
    """
    Retrieve player statistics including cards, goals, assists, and minutes played.

    Args:
        player_id (str): Player ID.
        cards_data (pd.DataFrame): DataFrame containing card data.
        stats_data (pd.DataFrame): DataFrame containing player statistics.

    Returns:
        Dict[str, int]: Player statistics.
    """
    stats = {
        "no_Yellow_cards": 0,
        "no_Red_cards": 0,
        "no_Goals": 0,
        "no_Assists": 0,
        "Minutes_Played": 0
    }

    card_data = cards_data[cards_data['Player_id'] == player_id]
    if not card_data.empty:
        stats.update({
            "no_Yellow_cards": int(card_data['No_of_Yellow_cards'].values[0]),
            "no_Red_cards": int(card_data['No_of_Red_cards'].values[0])
        })

    stat_data = stats_data[stats_data['Player_id'] == player_id]
    if not stat_data.empty:
        stats.update({
            "no_Goals": int(stat_data['Goals'].values[0]),
            "no_Assists": int(stat_data['Assists'].values[0]),
            "Minutes_Played": int(stat_data['Minutes_Played'].values[0])
        })

    return stats

def build_country_json(data: Dict[str, pd.DataFrame]) -> List[CountryData]:
    """
    Build the country JSON data structure.

    Args:
        data (Dict[str, pd.DataFrame]): Dictionary of DataFrames.

    Returns:
        List[CountryData]: List of country data dictionaries.
    """
    countries_json = []

    for _, row in data['Country'].iterrows():
        country_name = row['Country_Name']

        players = data['Players'][data['Players']['Country'] == country_name].to_dict(orient='records')
        for player in players:
            player.update(get_player_stats(player['Player_id'], data['Player_Cards'], data['Player_Assists_Goals']))

        world_cup_history = data['Worldcup_History'][data['Worldcup_History']['Winner'] == country_name].to_dict(orient='records')

        countries_json.append({
            "Country": country_name,
            "Capital": row['Capital'],
            "Population": format_population(row['Population']),
            "Manager": row['Manager'],
            "Players": players,
            "World_Cup_History": world_cup_history
        })

    return countries_json

def build_stadium_json(match_data: pd.DataFrame) -> List[StadiumData]:
    """
    Build the stadium JSON data structure.

    Args:
        match_data (pd.DataFrame): DataFrame containing match results.

    Returns:
        List[StadiumData]: List of stadium data dictionaries.
    """
    stadium_json = []

    stadium_groups = match_data.groupby(['Stadium', 'Host_city'])

    for (stadium, city), match_group in stadium_groups:
        matches = match_group[['Team1', 'Team2', 'Team1Score', 'Team2Score', 'Date']].to_dict(orient='records')

        stadium_json.append({
            "Stadium": stadium,
            "City": city,
            "Matches": matches
        })

    return stadium_json

def save_json_data(country_json: List[CountryData], stadium_json: List[StadiumData]) -> None:
    """
    Save JSON data to files.

    Args:
        country_json (List[CountryData]): List of country data dictionaries.
        stadium_json (List[StadiumData]): List of stadium data dictionaries.
    """
    with open('country_data_output.json', 'w') as country_file:
        json.dump(country_json, country_file, indent=4)

    with open('stadium_data_output.json', 'w') as stadium_file:
        json.dump(stadium_json, stadium_file, indent=4)

    print("Data processing complete! JSON files generated.")

def main() -> None:
    """
    Main execution function.
    """
    # Load all data
    data = load_all_data()

    # Assign column names
    assign_column_names(data)

    # Build JSON structures
    country_json = build_country_json(data)
    stadium_json = build_stadium_json(data['Match_results'])

    # Save JSON data
    save_json_data(country_json, stadium_json)

if __name__ == "__main__":
    main()